In [1]:
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

import os
import h5py
import librosa
import itertools
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import scipy

import keras
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
def split_song(X, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    shape = X.shape[0]
    chunk = int(shape*window)
    offset = int(chunk*(1.-overlap))
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, shape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)

    return np.array(temp_X)

In [3]:
def extract_melspectrogram(songs):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft = 1024, hop_length = 512)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    return np.array(list(tsongs))

In [4]:
def read_data(src_dir, song_samples):
    # Empty array of dicts with the processed features from all files
    arr_specs = []
    arr_names = []

    # Read files from the folders
    for root, subdirs, files in os.walk(src_dir):
        for file in files:
            # Read the audio file
            file_name = src_dir + "/" + file
            print("Read file: {}".format(file_name))

            signal, sr = librosa.load(file_name, offset=30.0, duration=30.0)
            signal = signal[:song_samples]

            # Convert to dataset of spectograms/melspectograms
            signals = split_song(signal)

            # Convert to "spec" representation
            specs = extract_melspectrogram(signals)
            # Save files
            arr_specs.append(specs)
            arr_names.append(file_name)
    return np.array(arr_specs), np.array(arr_names)

In [5]:
def predict_genres(songs, models_name):

    model = load_model(models_name)

    predicts = []
    for X in songs:
        predicts.append(model.predict(X))

    return np.array(predicts)

In [6]:
def normalize_data(predicts):

    songs = []
    
    #normalize the results
    for song_predict in predicts:
        i = 0
        temp_separate = np.zeros((10, 19))
        for sample in song_predict:
            j = 0
            for x in sample:
                temp_separate[j][i] = x
                j = j + 1
            i = i + 1
        
        genres_temp = []
        for x in temp_separate:
            genres_temp.append(scipy.mean(x))
            
        songs.append(genres_temp)

    return np.array(songs)

In [7]:
song_samples = 660000
test_folder = 'data/test'
songs_sample, file_names = read_data(test_folder, song_samples)
print(songs_sample.shape)


Read file: data/test/Anakelly - Mad World.mp3
Read file: data/test/Imagine Dragons - Natural.mp3
Read file: data/test/Lordi - Hard Rock Hallelujah (Евровидение 2006 Финляндия).mp3
Read file: data/test/Nirvana - Lithium .mp3
Read file: data/test/Omfg_-_Hello_-_Omfg_-_Hello(uzimusic.ru).mp3
Read file: data/test/Queen - We Are The Champions.mp3
Read file: data/test/the-animals-the-house-of-the-rising-sun.mp3
Read file: data/test/Toni Esposito - Kalimba de luna.mp3
Read file: data/test/Ария - Город.mp3
Read file: data/test/Би-2 - Полковнику Никто Не Пишет.mp3
(10, 19, 128, 129, 1)


In [8]:
models_name = 'models/model_snn.h5'
predicts = predict_genres(songs_sample, models_name)
print(predicts.shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
(10, 19, 10)


In [31]:
import pandas as pd 


genres = ['metal', 'disco', 'classical', 'hiphop', 'jazz', 
           'country', 'pop', 'blues', 'reggae', 'rock']

songs = normalize_data(predicts)
df = pd.DataFrame(data=songs,
                  index=file_names,
                  columns=genres
                 )
df

,metal,disco,classical,hiphop,jazz,country,pop,blues,reggae,rock
data/test/Anakelly - Mad World.mp3,2.617636e-03,0.027423,8.764359e-02,0.005194,7.048850e-01,4.472632e-02,0.016934,2.044512e-02,0.013695,7.643739e-02
data/test/Imagine Dragons - Natural.mp3,2.062825e-02,0.028898,1.364613e-01,0.150056,4.865083e-02,1.407836e-01,0.207258,1.110660e-01,0.011877,1.443206e-01
data/test/Lordi - Hard Rock Hallelujah (Евровидение 2006 Финляндия).mp3,7.029547e-01,0.004904,1.948280e-06,0.003073,2.704055e-06,1.042507e-03,0.034970,1.080310e-05,0.000133,2.529070e-01
data/test/Nirvana - Lithium .mp3,4.235891e-01,0.016226,6.173540e-07,0.021857,9.187451e-03,6.524647e-02,0.111898,1.442424e-02,0.045913,2.916577e-01
data/test/Omfg_-_Hello_-_Omfg_-_Hello(uzimusic.ru).mp3,2.800467e-05,0.001287,2.453561e-23,0.076270,3.041532e-09,3.217711e-08,0.886027,9.309575e-06,0.036368,1.094348e-05
data/test/Queen - We Are The Champions.mp3,7.575808e-02,0.133373,9.809195e-04,0.022238,1.042615e-02,1.835874e-01,0.168490,2.993361e-03,0.016645,3.855079e-01
data/test/the-animals-the-house-of-the-rising-sun.mp3,8.656626e-03,0.024994,3.093279e-02,0.002034,2.977208e-01,2.703657e-01,0.097997,3.677638e-02,0.008612,2.219109e-01
data/test/Toni Esposito - Kalimba de luna.mp3,1.475260e-07,0.932563,2.191883e-25,0.000551,6.756444e-19,1.246563e-07,0.000388,6.362338e-09,0.066496,9.078239e-07
data/test/Ария - Город.mp3,3.788589e-01,0.020815,5.807119e-02,0.014737,3.066122e-04,2.436819e-02,0.050803,4.702854e-03,0.002886,4.444516e-01
data/test/Би-2 - Полковнику Никто Не Пишет.mp3,9.860520e-01,0.000108,1.251293e-12,0.000532,4.715566e-08,1.869529e-06,0.000414,1.063688e-06,0.000003,1.288798e-02


In [26]:
df.to_csv('songs.csv')

In [27]:
dd = pd.read_csv('songs.csv', sep=',')
dd

,Unnamed: 0,metal,disco,classical,hiphop,jazz,country,pop,blues,reggae,rock
0,data/test/Anakelly - Mad World.mp3,0.002618,0.020628,0.702955,4.235891e-01,2.800467e-05,0.075758,0.008657,1.475260e-07,0.378859,9.860520e-01
1,data/test/Imagine Dragons - Natural.mp3,0.027423,0.028898,0.004904,1.622584e-02,1.286506e-03,0.133373,0.024994,9.325629e-01,0.020815,1.080955e-04
2,data/test/Lordi - Hard Rock Hallelujah (Еврови...,0.087644,0.136461,0.000002,6.173540e-07,2.453561e-23,0.000981,0.030933,2.191883e-25,0.058071,1.251293e-12
3,data/test/Nirvana - Lithium .mp3,0.005194,0.150056,0.003073,2.185727e-02,7.627021e-02,0.022238,0.002034,5.513842e-04,0.014737,5.323910e-04
4,data/test/Omfg_-_Hello_-_Omfg_-_Hello(uzimusic...,0.704885,0.048651,0.000003,9.187451e-03,3.041532e-09,0.010426,0.297721,6.756444e-19,0.000307,4.715566e-08
5,data/test/Queen - We Are The Champions.mp3,0.044726,0.140784,0.001043,6.524647e-02,3.217711e-08,0.183587,0.270366,1.246563e-07,0.024368,1.869529e-06
6,data/test/the-animals-the-house-of-the-rising-...,0.016934,0.207258,0.034970,1.118982e-01,8.860269e-01,0.168490,0.097997,3.880878e-04,0.050803,4.140341e-04
7,data/test/Toni Esposito - Kalimba de luna.mp3,0.020445,0.111066,0.000011,1.442424e-02,9.309575e-06,0.002993,0.036776,6.362338e-09,0.004703,1.063688e-06
8,data/test/Ария - Город.mp3,0.013695,0.011877,0.000133,4.591317e-02,3.636809e-02,0.016645,0.008612,6.649645e-02,0.002886,2.560256e-06
9,data/test/Би-2 - Полковнику Никто Не Пишет.mp3,0.076437,0.144321,0.252907,2.916577e-01,1.094348e-05,0.385508,0.221911,9.078239e-07,0.444452,1.288798e-02
